<a href="https://colab.research.google.com/github/esequielsantos/ediBot/blob/develop/EdiBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Instalar Gemini

---



In [91]:
!pip install -U google-generativeai

## Set up my API key

To use the Gemini API, you'll need an API key. If you don't already have one, create a key in Google AI Studio.

<a class="button" href="https://aistudio.google.com/app/apikey" target="_blank" rel="noopener noreferrer">Get an API key</a>

In Colab, add the key to the secrets manager under the "🔑" in the left panel. Give it the name `GOOGLE_API_KEY`. Then pass the key to the SDK:


In [92]:
import google.generativeai as genai
# Used to securely store your API key
from google.colab import userdata

GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

Listar modelos disponiveis


In [93]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

KeyboardInterrupt: 

Configurando o modelo
Temperaura:

In [94]:
gen_config = {
    "candidate_count": 1,
    "temperature": 1,
}

Segurança:

In [95]:
saf_settings = {
    "HARASSMENT": "BLOCK_NONE",
    "HATE": "BLOCK_NONE",
    "SEXUAL": "BLOCK_NONE",
    "DANGEROUS": "BLOCK_NONE",
}

Usar **modelo** escolhido acima

In [96]:
model = genai.GenerativeModel(model_name='gemini-1.0-pro-latest', safety_settings= saf_settings, generation_config= gen_config)

#Enviando mensagem ao modelo

In [ ]:
response = model.generate_content("Estou programndo em Phyton. Sugira 5 opções indispensáveis de conhecermos no aprendizado de Phyton")
#print(model.generate_content("Estou programndo em Phyton. Sugira 5 opções indispensáveis de conhecermos no aprendizado de Phyton").text)

##Exibindo Resposta

In [97]:
print(response.text)

NameError: name 'response' is not defined

#EdiBOT

> Adicionar aspas INICIANDO O EdiBot

**Iniciando**



In [98]:
from datetime import datetime

In [99]:
chat = model.start_chat(history=[])

In [106]:
def edibot(prompt, history):
  if not chat.history:
    agora = datetime.now()
    hora = agora.hour
    periodo_dia = ""
    if 6 <= hora < 12:
      periodo_dia = "manhã"
    elif 12 <= hora < 18:
      periodo_dia = "tarde"
    else:
      periodo_dia = "noite"
    bemvindo = model.generate_content(f"Você é um atendente de uma Loja física de informatica chamada ByteMe, que vende computadores, laptops e acessórios chamado EdiBot. Escreva uma mensagem de saudação de {periodo_dia} ").text
    response = chat.send_message(prompt)
    return agora + '\n' + bemvindo + '\n\n-----------------------\n' + response.text
  else:
    prompt = prompt.lower()
    agradecer = model.generate_content("Você é um atendente de uma empresa de informatica chamada ByteMe. Escreva uma mensagem curta de agradecimento por ter entrado em contato").text
    if prompt == "history":
      return chat.history
    elif 'obrigado' in prompt:
      return agradecer
    elif 'tchau' in prompt:
      return agradecer
    elif 'fim' in prompt:
      return agradecer
    elif 'sair' in prompt:
      return agradecer
    else:
      response = chat.send_message(prompt)
      return response.text
  continuar = model.generate_content("Escreva uma mensagem curta e agradavél perguntando se deseja mais alguma informação").text
  return continuar

##Teste da funcao inteligente
*Exige parada total do GRadio*

In [107]:
print(edibot("Bom dia, preciso de um computador novo para jogar", []))

ReadTimeout: HTTPConnectionPool(host='localhost', port=37431): Read timed out. (read timeout=60.0)

In [ ]:
print(edibot("e se for computador novo para jogar", []))

In [ ]:
print(edibot("e se for pra usar Autocad ?", []))

In [ ]:
#print(edibot('history', []))
chat.history

##Uso do GRADIO

In [ ]:
!pip install -U gradio

In [ ]:
import gradio as gr

#Tela Gráfica para o BOT

In [ ]:
demo = gr.ChatInterface(edibot,
                 title='BatePapo BiteMe',
                 textbox=gr.Textbox(placeholder="Pergunte ao EdiBOT"),
                 retry_btn='Voltar',
                 clear_btn='Limpar',
                 undo_btn='Desfazer',
                 stop_btn='Parar',
                 submit_btn='Enviar [Enter]').queue()

if __name__ == "__main__":
    demo.launch(debug=True)

Subir para o GitHub